# packages

In [1]:
#dependencies and setup
import pandas as pd
pd.options.display.float_format = '{:,.2f}'.format
import os
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns 
import plotly.express as px
from openpyxl import Workbook
import numpy as np
import requests
import json
import re
from pprint import pprint
import time
# SQLite dependencies
import sqlite3
from sqlalchemy import create_engine
from sqlalchemy import Column, Integer, String, Float
from pandas_profiling import ProfileReport

In [2]:
#  SQLite DB creation and establishing connection
database_path = "NJ_County_DB.sqlite"
engine = create_engine(f"sqlite:///{database_path}", echo=True)
sqlite_connection = engine.connect()

In [3]:
from sqlalchemy import text

In [6]:
sql_query = """SELECT name FROM sqlite_master  
  WHERE type='table';"""
tbls= pd.read_sql(sql_query,sqlite_connection)
tbls

2023-03-22 13:45:51,029 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("SELECT name FROM sqlite_master  
  WHERE type='table';")
2023-03-22 13:45:51,037 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-03-22 13:45:51,041 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("SELECT name FROM sqlite_master  
  WHERE type='table';")
2023-03-22 13:45:51,043 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-03-22 13:45:51,046 INFO sqlalchemy.engine.Engine SELECT name FROM sqlite_master  
  WHERE type='table';
2023-03-22 13:45:51,047 INFO sqlalchemy.engine.Engine [raw sql] ()


,name
0,nj_property_tax
1,nj_mortgage_rates
2,nj_population
3,nj_zillow_house_value_index
4,nj_food_desert
5,nj_poverty_median_income
6,nj_crime_detail
7,nj_school_performance
8,nj_zillow_observed_rent_index
9,nj_adi


# D3 dash

In [7]:
school = "nj_school_performance"
poverty = "nj_poverty_median_income"
crime = "nj_crime_detail"
population = "nj_population"
tax='nj_property_tax'

In [16]:
query = '''SELECT POV.county_name, POV.median_hh_income, POV.poverty_rate, POV.county_code,
           CRM.total AS total_offense, ROUND(CRM2.total,2) AS rate_per_100k, CRM3.total AS total_arrest,
           POP.population, SCH.school_score, tax_rate
           FROM nj_poverty_median_income AS POV 
           
           INNER JOIN 
           (SELECT county_name, sum(total) as total 
           FROM nj_crime_detail 
           WHERE report_type = 'Number of Offenses' AND year =2020
           GROUP BY county_name) AS CRM 
           ON POV.county_name = CRM.county_name 
           
           INNER JOIN 
           (SELECT county_name, sum(total) as total 
           FROM nj_crime_detail 
           WHERE report_type = 'Rate Per 100,000' AND year =2020
           GROUP BY county_name) AS CRM2 
           ON POV.county_name = CRM2.county_name
           
           INNER JOIN 
           (SELECT county_name, sum(total) as total 
           FROM nj_crime_detail 
           WHERE report_type = 'Number of Arrests' AND year =2020
           GROUP BY county_name) AS CRM3 
           ON POV.county_name = CRM3.county_name
           INNER JOIN 
           (SELECT county_name, SUM(est_pop) AS population 
           FROM nj_population WHERE year =2021 GROUP BY county_name) AS POP 
           ON POV.county_name = POP.county_name
           INNER JOIN 
           (SELECT county_name, ROUND(AVG(score),2) AS school_score 
           FROM nj_school_performance WHERE year =2020 GROUP BY county_name) AS SCH 
           ON POV.county_name = SCH.county_name
           INNER JOIN 
           (SELECT county_name, AVG(tax_rate) as tax_rate
           FROM nj_property_tax WHERE year =2022 GROUP BY county_name) AS TAX ON POV.county_name = TAX.county_name
           WHERE POV.year = 2021'''
df = pd.read_sql(text(query), sqlite_connection)
df.head()

2023-03-22 14:02:28,762 INFO sqlalchemy.engine.Engine SELECT POV.county_name, POV.median_hh_income, POV.poverty_rate, POV.county_code,
           CRM.total AS total_offense, ROUND(CRM2.total,2) AS rate_per_100k, CRM3.total AS total_arrest,
           POP.population, SCH.school_score, tax_rate
           FROM nj_poverty_median_income AS POV 
           
           INNER JOIN 
           (SELECT county_name, sum(total) as total 
           FROM nj_crime_detail 
           WHERE report_type = 'Number of Offenses' AND year =2020
           GROUP BY county_name) AS CRM 
           ON POV.county_name = CRM.county_name 
           
           INNER JOIN 
           (SELECT county_name, sum(total) as total 
           FROM nj_crime_detail 
           WHERE report_type = 'Rate Per 100,000' AND year =2020
           GROUP BY county_name) AS CRM2 
           ON POV.county_name = CRM2.county_name
           
           INNER JOIN 
           (SELECT county_name, sum(total) as total 
           FRO

,county_name,median_hh_income,poverty_rate,county_code,total_offense,rate_per_100k,total_arrest,population,school_score,tax_rate
0,ATLANTIC,65693,15.10,001,"5,719.00","37,746.00","1,384.00",274966,34.88,3.01
1,BERGEN,104777,7.50,003,"7,694.00","181,510.20","1,698.00",953819,65.40,2.55
2,BURLINGTON,94043,7.60,005,"5,184.00","36,120.30","1,374.00",464269,49.80,3.08
3,CAMDEN,77675,12.00,007,"11,545.00","85,374.00","2,798.00",523771,37.70,4.29
4,CAPE MAY,74896,9.00,009,"2,010.00","61,197.40",369.00,95661,48.98,1.43


# plotly dash

In [39]:
query = '''
SELECT T1.*, T2.population, T3.tax_rate
FROM 
(SELECT county_name,median_hh_income,poverty_rate 
FROM nj_poverty_median_income WHERE year=2021) AS T1
INNER JOIN 
(SELECT county_name, SUM(est_pop) as population 
FROM nj_population WHERE year=2021 GROUP BY county_name) AS T2
ON T1.county_name = T2.county_name
INNER JOIN 
(SELECT county_name, AVG(tax_rate) as tax_rate 
FROM nj_property_tax WHERE year=2022 GROUP BY county_name) AS T3
ON T1.county_name = T3.county_name;
'''
metadata_df = pd.read_sql_query(query, sqlite_connection)
metadata_dict = metadata_df.to_dict(orient='records')

query='''
SELECT county_name, school, score FROM nj_school_performance WHERE year=2020 ORDER BY county_name, score DESC;
'''
school_df = pd.read_sql_query(query, sqlite_connection)
school_dict = school_df.to_dict(orient='records')

query='''
SELECT * FROM nj_crime_detail WHERE year=2020 AND report_type = 'Number of Offenses'
'''
crime_df = pd.read_sql_query(query, sqlite_connection)
crime_dict = crime_df.to_dict(orient='records')

data_json= {}
data_json["metadata"] = metadata_dict
data_json["school"] = school_dict
data_json["crime"] = crime_dict
with open("static/data/plotly.json", "w",encoding ='utf8') as outfile:  
    json.dump(data_json, outfile, indent = 1) 
print("Data retrieval successfull")

print("Data retrieval successfull")

2023-03-22 10:26:34,972 INFO sqlalchemy.engine.Engine 
SELECT T1.*, T2.population, T3.tax_rate
FROM 
(SELECT county_name,median_hh_income,poverty_rate 
FROM nj_poverty_median_income WHERE year=2021) AS T1
INNER JOIN 
(SELECT county_name, SUM(est_pop) as population 
FROM nj_population WHERE year=2021 GROUP BY county_name) AS T2
ON T1.county_name = T2.county_name
INNER JOIN 
(SELECT county_name, AVG(tax_rate) as tax_rate 
FROM nj_property_tax WHERE year=2022 GROUP BY county_name) AS T3
ON T1.county_name = T3.county_name;

2023-03-22 10:26:34,975 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-03-22 10:26:34,991 INFO sqlalchemy.engine.Engine 
SELECT county_name, school, score FROM nj_school_performance WHERE year=2020 ORDER BY county_name, score DESC;

2023-03-22 10:26:34,992 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-03-22 10:26:35,023 INFO sqlalchemy.engine.Engine 
SELECT * FROM nj_crime_detail WHERE year=2020 AND report_type = 'Number of Offenses'

2023-03-22 10:26:35,024 INFO sql

In [40]:
data_json

{'metadata': [{'county_name': 'ATLANTIC',
   'median_hh_income': 65693,
   'poverty_rate': 15.1,
   'population': 274966,
   'tax_rate': 3.0129130434782607},
  {'county_name': 'BERGEN',
   'median_hh_income': 104777,
   'poverty_rate': 7.5,
   'population': 953819,
   'tax_rate': 2.5501714285714283},
  {'county_name': 'BURLINGTON',
   'median_hh_income': 94043,
   'poverty_rate': 7.6,
   'population': 464269,
   'tax_rate': 3.0822749999999997},
  {'county_name': 'CAMDEN',
   'median_hh_income': 77675,
   'poverty_rate': 12.0,
   'population': 523771,
   'tax_rate': 4.288166666666665},
  {'county_name': 'CAPE MAY',
   'median_hh_income': 74896,
   'poverty_rate': 9.0,
   'population': 95661,
   'tax_rate': 1.4321249999999999},
  {'county_name': 'CUMBERLAND',
   'median_hh_income': 58030,
   'poverty_rate': 14.4,
   'population': 153627,
   'tax_rate': 3.2979999999999996},
  {'county_name': 'ESSEX',
   'median_hh_income': 66191,
   'poverty_rate': 15.1,
   'population': 854917,
   'tax_r

# School

In [27]:
query = '''SELECT * FROM nj_school_performance WHERE year=2020 ORDER BY county_name,district,grades,school;'''
test = pd.read_sql_query(query, sqlite_connection)

print("Query successfull")

data_json = {}
data_json["name"] = "school"
data_json["description"] = "school"

counties = list(test['county_name'].unique())

children = []
for i in range(len(counties)):
    child1 = {}
    child1["name"] = counties[i]
    child1["description"] = test['score'].loc[test['county_name']==counties[i]].mean()
    district = list(test['district'].loc[test['county_name']==counties[i]].unique())
    child2_list = []
    for k in range(len(district)):
        child2 = {}
        child2["name"] = district[k]
        child2["description"] = test['score'].loc[(test['county_name']==counties[i]) & \
                                                  (test['district'] == district[k])].mean()
        child3_list = []
        gradespan = list(test['grades'].loc[(test['county_name']==counties[i]) &\
                                               (test['district'] == district[k])].unique())
        for j in range(len(gradespan)):
            child3 = {}
            child3["name"] = gradespan[j]
            child3["description"] = test["score"].loc[(test['county_name']==counties[i]) & \
                                                      (test['district'] == district[k]) & \
                                                      (test['grades'] == gradespan[j])].mean()
            child4_list = []
            for index,row in test.loc[(test['county_name']==counties[i]) & \
                                      (test['district'] == district[k]) & \
                                      (test['grades'] == gradespan[j])].iterrows():
                child4 = {}
                child4["name"] = row["school"]
                child4["description"] = row["score"]
                child4["size"] = row["score"]
                child4_list.append(child4)
            child3["children"] = child4_list
            child3_list.append(child3)
        child2["children"] = child3_list
        child2_list.append(child2)
    child1["children"] = child2_list
    children.append(child1)

data_json["children"] = children
with open("static/data/school.json", "w",encoding ='utf8') as outfile:  
    json.dump(data_json, outfile, indent = 1) 
print("Data retrieval successfull")
if data_json:
    print("Json ready")
else:
    print("Json failed!")

2023-03-22 10:13:17,419 INFO sqlalchemy.engine.Engine SELECT * FROM nj_school_performance WHERE year=2020 ORDER BY county_name,district,grades,school;
2023-03-22 10:13:17,421 INFO sqlalchemy.engine.Engine [raw sql] ()
Query successfull
Data retrieval successfull
Json ready


In [28]:
data_json

{'name': 'school',
 'description': 'school',
 'children': [{'name': 'ATLANTIC',
   'description': 34.875757575757575,
   'children': [{'name': 'Absecon Public Schools District',
     'description': 38.55,
     'children': [{'name': '5-8',
       'description': 49.7,
       'children': [{'name': 'Emma C Attales',
         'description': 49.7,
         'size': 49.7}]},
      {'name': 'PK, KG-4',
       'description': 27.4,
       'children': [{'name': 'H Ashton Marsh',
         'description': 27.4,
         'size': 27.4}]}]},
    {'name': 'Atlantic City School District',
     'description': 15.666666666666666,
     'children': [{'name': 'K-8',
       'description': 16.1,
       'children': [{'name': 'Texas Avenue School',
         'description': 13.5,
         'size': 13.5},
        {'name': 'Texas Avenue School', 'description': 18.7, 'size': 18.7}]},
      {'name': 'PK, KG-5',
       'description': 17.9,
       'children': [{'name': 'Brighton Avenue School',
         'description': 17.9

# Crime

In [8]:
query = """
SELECT * FROM nj_crime_detail WHERE report_type = 'Number of Offenses' AND total <> 0 AND year=2020;
"""
test = pd.read_sql_query(query, sqlite_connection)
print("Query successfull")

test=test.drop(['year','population'],axis=1)\
            .groupby(['county_name','agency'],as_index=False).sum()
test = pd.melt(test,\
        id_vars=['county_name','agency'], value_vars=['murder',
       'rape', 'robbery', 'assault', 'burglary', 'larceny', 'auto_theft'],
        var_name='crime_type', value_name='rate').sort_values(['county_name','agency','crime_type'],ignore_index=True)

data_json = {}
data_json["name"] = "crime"
data_json["description"] = "crime"

counties = list(test['county_name'].unique())

children = []
children = []
for i in range(len(counties)):
    child1 = {}
    child1["name"] = counties[i]
    child1["description"] = test['rate'].loc[test['county_name']==counties[i]].mean()
    police_dept = list(test['agency'].loc[test['county_name']==counties[i]].unique())
    child2_list = []
    for k in range(len(police_dept)):
        child2 = {}
        child2["name"] = police_dept[k]
        child2["description"] = test['rate'].loc[(test['county_name']==counties[i]) &\
                                                  (test['agency'] == police_dept[k])].mean()
        child3_list = []
        crime_type = list(test['crime_type'].loc[(test['county_name']==counties[i]) &\
                                                 (test['agency'] == police_dept[k])].unique())
        for j in range(len(crime_type)):
            for index,row in test.loc[(test['county_name']==counties[i]) &\
                                      (test['agency'] == police_dept[k]) &\
                                      (test['crime_type'] == crime_type[j])].iterrows():
                child3 = {}
                child3["name"] = row["crime_type"]
                child3["description"] = row["rate"]
                child3["size"] = row["rate"]
                child3_list.append(child3)
            child2["children"] = child3_list
        child2_list.append(child2)
    child1["children"] = child2_list
    children.append(child1)
data_json["children"] = children
with open("../static/data/crime.json", "w",encoding ='utf8') as outfile:  
    json.dump(data_json, outfile, indent = 1) 
print("Data retrieval successfull")
if data_json:
    print("Json ready")
else:
    print("Json failed!")

2023-03-22 19:56:05,298 INFO sqlalchemy.engine.Engine 
SELECT * FROM nj_crime_detail WHERE report_type = 'Number of Offenses' AND total <> 0 AND year=2020;

2023-03-22 19:56:05,300 INFO sqlalchemy.engine.Engine [raw sql] ()
Query successfull
Data retrieval successfull
Json ready


In [9]:
data_json

{'name': 'crime',
 'description': 'crime',
 'children': [{'name': 'ATLANTIC',
   'description': 43.0,
   'children': [{'name': 'ABSECON CITY PD',
     'description': 52.857142857142854,
     'children': [{'name': 'assault', 'description': 18.0, 'size': 18.0},
      {'name': 'auto_theft', 'description': 17.0, 'size': 17.0},
      {'name': 'burglary', 'description': 44.0, 'size': 44.0},
      {'name': 'larceny', 'description': 284.0, 'size': 284.0},
      {'name': 'murder', 'description': 0.0, 'size': 0.0},
      {'name': 'rape', 'description': 2.0, 'size': 2.0},
      {'name': 'robbery', 'description': 5.0, 'size': 5.0}]},
    {'name': 'ATLANTIC CITY PD',
     'description': 188.85714285714286,
     'children': [{'name': 'assault', 'description': 156.0, 'size': 156.0},
      {'name': 'auto_theft', 'description': 89.0, 'size': 89.0},
      {'name': 'burglary', 'description': 139.0, 'size': 139.0},
      {'name': 'larceny', 'description': 785.0, 'size': 785.0},
      {'name': 'murder', 'd

# Tax

In [29]:
query = '''SELECT DISTINCT * FROM nj_property_tax WHERE year =2022'''
test = pd.read_sql_query(query, sqlite_connection)
print("Query successfull")

data_json = {}
data_json["name"] = "tax"
data_json["description"] = "tax"

counties = list(test['county_name'].unique())

children = []
for i in range(len(counties)):
    child1 = {}
    child1["name"] = counties[i]
    child1["description"] = test['tax_rate'].loc[test['county_name']==counties[i]].mean()
    district = list(test['district_name'].loc[test['county_name']==counties[i]].unique())
    child2_list = []
    for k in range(len(district)):
        for index,row in test.loc[(test['county_name']==counties[i]) & (test['district_name'] == district[k])].iterrows():
            child2 = {}
            child2["name"] = row["district_name"]
            child2["description"] = row["tax_rate"]
            child2["size"] = row["tax_rate"]
            child2_list.append(child2)
    child1["children"] = child2_list
    children.append(child1)
data_json["children"] = children
with open("static/data/tax_sunburst.json", "w",encoding ='utf8') as outfile:  
    json.dump(data_json, outfile, indent = 1) 
# print("Data retrieval successfull")
# if data_json:
#     print("Json ready")
# else:
#     print("Json failed!")


2023-03-22 10:14:53,236 INFO sqlalchemy.engine.Engine SELECT DISTINCT * FROM nj_property_tax WHERE year =2022
2023-03-22 10:14:53,243 INFO sqlalchemy.engine.Engine [raw sql] ()
Query successfull


In [30]:
data_json

{'name': 'tax',
 'description': 'tax',
 'children': [{'name': 'ATLANTIC',
   'description': 3.0129130434782607,
   'children': [{'name': 'ABSECON CITY', 'description': 3.306, 'size': 3.306},
    {'name': 'ATLANTIC CITY CITY',
     'description': 3.7769999999999997,
     'size': 3.7769999999999997},
    {'name': 'BRIGANTINE CITY',
     'description': 1.7369999999999999,
     'size': 1.7369999999999999},
    {'name': 'BUENA BORO', 'description': 3.242, 'size': 3.242},
    {'name': 'BUENA VISTA TWP', 'description': 2.762, 'size': 2.762},
    {'name': 'CORBIN CITY CITY', 'description': 2.017, 'size': 2.017},
    {'name': 'EGG HARBOR CITY', 'description': 5.453, 'size': 5.453},
    {'name': 'EGG HARBOR TWP', 'description': 3.335, 'size': 3.335},
    {'name': 'ESTELL MANOR CITY',
     'description': 2.7439999999999998,
     'size': 2.7439999999999998},
    {'name': 'FOLSOM BORO', 'description': 2.163, 'size': 2.163},
    {'name': 'GALLOWAY TWP', 'description': 3.211, 'size': 3.211},
    {'na

# GeoJSON

In [41]:
geojson_url = 'https://opendata.arcgis.com/datasets/5f45e1ece6e14ef5866974a7b57d3b95_1.geojson'
geojson = requests.get(geojson_url).json()

In [42]:
geojson.keys()

dict_keys(['type', 'name', 'crs', 'features'])

In [43]:
geojson['features'][0].keys()

dict_keys(['type', 'properties', 'geometry'])

In [44]:
geojson['features'][0]['geometry'].keys()

dict_keys(['type', 'coordinates'])

In [45]:
geojson['features'][0]['properties']['COUNTY']

'ATLANTIC'

In [54]:
query = '''SELECT T1.*, T2.school_score, T3.tax_rate, T4.poverty_rate, T5.median_hh_income, T6.population
        FROM
        (SELECT county_name, sum(total) as crime_rate 
        FROM nj_crime_detail WHERE report_type = 'Rate Per 100,000' AND year=2020 GROUP BY 1) AS T1
        INNER JOIN 
        (SELECT county_name, AVG(score) AS school_score 
        FROM nj_school_performance WHERE year=2020 GROUP BY 1) AS T2 
        ON T1.county_name = T2.county_name
        INNER JOIN (SELECT county_name, AVG(tax_rate) AS tax_rate 
        FROM nj_property_tax WHERE year=2022 GROUP BY 1) AS T3
        ON T1.county_name = T3.county_name
        INNER JOIN 
        (SELECT county_name, AVG(poverty_rate) AS poverty_rate 
        FROM nj_poverty_median_income WHERE year=2021 GROUP BY 1) AS T4
        ON T1.county_name = T4.county_name
        INNER JOIN 
        (SELECT county_name, AVG(median_hh_income) AS median_hh_income 
        FROM nj_poverty_median_income WHERE year=2021 GROUP BY 1) AS T5
        ON T1.county_name = T5.county_name
        INNER JOIN 
        (SELECT county_name, SUM(est_pop) AS population 
        FROM nj_population WHERE year=2021 GROUP BY 1) AS T6
        ON T1.county_name = T6.county_name
'''

In [55]:
test = pd.read_sql_query(query, sqlite_connection)
test.head()

2023-03-22 11:05:00,196 INFO sqlalchemy.engine.Engine SELECT T1.*, T2.school_score, T3.tax_rate, T4.poverty_rate, T5.median_hh_income, T6.population
        FROM
        (SELECT county_name, sum(total) as crime_rate 
        FROM nj_crime_detail WHERE report_type = 'Rate Per 100,000' AND year=2020 GROUP BY 1) AS T1
        INNER JOIN 
        (SELECT county_name, AVG(score) AS school_score 
        FROM nj_school_performance WHERE year=2020 GROUP BY 1) AS T2 
        ON T1.county_name = T2.county_name
        INNER JOIN (SELECT county_name, AVG(tax_rate) AS tax_rate 
        FROM nj_property_tax WHERE year=2022 GROUP BY 1) AS T3
        ON T1.county_name = T3.county_name
        INNER JOIN 
        (SELECT county_name, AVG(poverty_rate) AS poverty_rate 
        FROM nj_poverty_median_income WHERE year=2021 GROUP BY 1) AS T4
        ON T1.county_name = T4.county_name
        INNER JOIN 
        (SELECT county_name, AVG(median_hh_income) AS median_hh_income 
        FROM nj_poverty_media

,county_name,crime_rate,school_score,tax_rate,poverty_rate,median_hh_income,population
0,ATLANTIC,"37,746.00",34.88,3.01,15.10,"65,693.00",274966
1,BERGEN,"181,510.20",65.40,2.55,7.50,"104,777.00",953819
2,BURLINGTON,"36,120.30",49.80,3.08,7.60,"94,043.00",464269
3,CAMDEN,"85,374.00",37.70,4.29,12.00,"77,675.00",523771
4,CAPE MAY,"61,197.40",48.98,1.43,9.00,"74,896.00",95661


In [63]:
for i in range(len(geojson['features'])):
    county = geojson['features'][i]['properties']['COUNTY']
    geojson['features'][i]['properties']['crime_rate'] = \
    round(test['crime_rate'].loc[test['county_name']==county].item(),2)
    geojson['features'][i]['properties']['school_score'] = \
    round(test['school_score'].loc[test['county_name']==county].item(),2)
    geojson['features'][i]['properties']['tax_rate'] = \
    round(test['tax_rate'].loc[test['county_name']==county].item(),2)
    geojson['features'][i]['properties']['poverty_rate'] = \
    round(test['poverty_rate'].loc[test['county_name']==county].item(),2)
    geojson['features'][i]['properties']['median_hh_income'] = \
    round(test['median_hh_income'].loc[test['county_name']==county].item(),2)
    geojson['features'][i]['properties']['population'] = \
    test['population'].loc[test['county_name']==county].item()

In [66]:
geojson['features'][0]['properties']

{'OBJECTID': 1,
 'COUNTY': 'ATLANTIC',
 'COUNTY_LABEL': 'Atlantic County',
 'CO': 'ATL',
 'GNIS_NAME': 'County of Atlantic',
 'GNIS': '882270',
 'FIPSSTCO': '34001',
 'FIPSCO': '1',
 'ACRES': 390813.84096806,
 'SQ_MILES': 610.64662651,
 'POP2010': 274549,
 'POP2000': 252552,
 'POP1990': 275372,
 'POP1980': 204615,
 'POPDEN2010': 450,
 'POPDEN2000': 414,
 'POPDEN1990': 451,
 'POPDEN1980': 335,
 'REGION': 'COASTAL',
 'GLOBALID': '{9BB3C36A-5C8E-44F2-85B8-AAA00F4E7F59}',
 'Shape_Length': 650228.1539003241,
 'Shape_Area': 17023850912.569162,
 'POP2020': 274534,
 'POPDEN2020': 450,
 'crime_rate': 37746.0,
 'school_score': 34.88,
 'tax_rate': 3.01,
 'poverty_rate': 15.1,
 'median_hh_income': 65693.0,
 'population': 274966}

In [65]:
with open("static/data/final.geojson", "w",encoding ='utf8') as outfile:  
    json.dump(geojson, outfile) 